In [ ]:
from selenium import webdriver 
from time import sleep
from urllib.request import urlopen
from bs4 import BeautifulSoup
import traceback
import os
os.remove("place_email.csv")
print("Files Removed!")

target = "places"
keywords = "Walnut+small+business"

options = webdriver.ChromeOptions()
prefs = {
    'profile.default_content_setting_values' :
        {
        'notifications' : 2
         }
}
options.add_experimental_option('prefs',prefs)
options.add_argument("disable-infobars") 
options.add_argument('--hide-scrollbars')  
options.add_argument('blink-settings=imagesEnabled=false')  
options.add_argument('--headless')

driver = webdriver.Chrome(r"C:\Users\UNclePeach\AnacondaProjects\Social Media CRM Analysis\chromedriver.exe", options = options) 
driver.get('https://www.facebook.com/')
  
username_box = driver.find_element_by_id('email') 
username_box.send_keys("xxxxxxxxx") 

password_box = driver.find_element_by_id('pass') 
password_box.send_keys("xxxxxxxx") 

login_box = driver.find_element_by_id('loginbutton') 
login_box.click() 

driver.get("https://www.facebook.com/search/"+target+"/?q="+keywords)

while driver.find_element_by_tag_name('div'):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    Divs=driver.find_element_by_tag_name('div').text
    if 'End of Results' in Divs:
        print ("end of results")
        break       
    else:
        continue 
        
ff = open('email.csv','a')
places = driver.find_elements_by_tag_name('li')
for place in places:
    info = place.get_attribute("id")
    if "anchor" in info:
        placeid = info[0:info.index("a")]
        url="https://www.facebook.com/"+placeid+"/about/"
    try:
        page = urlopen(url)
        soup = BeautifulSoup(page,'html.parser')       
        for link in soup.find_all("a"):
            linkstring=link.get("href")
            if "mailto:" in linkstring:
                mailto = linkstring[7:len(linkstring)]
                print(mailto)
                ff.write(mailto) 
                ff.write('\n')
    except:
        f=open("log.txt",'a')  
        traceback.print_exc(file=f)  
        f.flush()  
        f.close()
    continue        

print("Place email export done!")

#with open('email.csv','r') as in_file, open('place_email.csv','w') as out_file:
    #seen = set() 
    #for line in in_file:
        #if line in seen: continue 

        #seen.add(line)
        #out_file.write(line)

#需要添加判断.csv文件不为空时执行操作！！！！！
from pandas import read_csv 
df = read_csv('email.csv')
newDF = df.drop_duplicates()
newDF.to_csv('place_email.csv', index = None, header = None)
        
ff.close()
driver.quit()
os.remove("email.csv")         
print("Place email clean up!")